<a href="https://colab.research.google.com/github/shaodwing/nmist-fashion/blob/Data-augmentation/Fashion_NMIST_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from tqdm import tqdm
from scipy import misc
import tensorflow as tf
from keras.datasets import fashion_mnist

#MobileNet by keras
from keras.applications.mobilenet import MobileNet
from keras.layers import Input,Dense,Dropout,Lambda
from keras.models import Model
from keras import backend as K


#MobileNetV2 by leonard
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

#Tiny  darknet
from keras import utils
from keras.callbacks import Callback
from keras.layers import BatchNormalization, Conv2D, GlobalAveragePooling2D, Activation, LeakyReLU, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD


#Load Data

In [0]:
np.random.seed(2019)
tf.set_random_seed(2019)

(X_train, Y_train ), (X_test, Y_test ) = fashion_mnist.load_data()

#Tiny dark net


In [19]:
#Used Joseph Redmon on tiny darknet to produce my work

batch_size = 256
num_classes = 10 
epochs = 40
img_cols = X_train.shape[1]
img_rows = X_train.shape[2]


if K.image_data_format() == 'channels_first':
  X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols);
  X_test  = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols);
  input_shape = (1, img_rows, img_cols);
else: 
  X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1);
  X_test  = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1);
  input_shape = (img_rows, img_cols, 1);


X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')
X_train /= 255
X_test  /= 255

Y_train = utils.to_categorical(Y_train, num_classes)
Y_test  = utils.to_categorical(Y_test, num_classes)

print(X_train.shape)
print(X_test.shape)


(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [0]:
#Data augmentation
imageGene = ImageDataGenerator(horizontal_flip=True, vertical_flip=True) #, rotation_range=90,zoom_range=0


In [0]:
#Function
def layerConv(output, filters, size, stride, pad, batch_normalization=True, activation='LeakyReLU'):
  output = Conv2D(kernel_size = (size,size), filters = filters, strides=stride, padding=pad)(output)
  if batch_normalization:
    output = BatchNormalization()(output);
  
  #Activation layer
  if activation=='LeakyReLU':
    output = LeakyReLU(alpha = 0.1)(output)
  else:
    output = Activation(activation)(output)
  
  return output

In [22]:
#LAYERS
#First set of convolution (9-12)
input_img = Input(shape=input_shape)
output = layerConv(output=input_img, filters=128, size=3, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')
output = layerConv(output=output   , filters=32 , size=1, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')
output = layerConv(output=output   , filters=256, size=3, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')
output = layerConv(output=output   , filters=32 , size=1, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')
output = layerConv(output=output   , filters=256, size=3, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')

#Max pooling to get 14x14 feature (13)
output = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format=None)(output)

#Second set of convolution (14-19)
output = layerConv(output=output, filters=64 , size=1, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')
output = layerConv(output=output, filters=256, size=3, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU') #512
output = layerConv(output=output, filters=64 , size=1, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')
output = layerConv(output=output, filters=256, size=3, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU') #512
output = layerConv(output=output, filters=64 , size=1, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU') #128

#Last layer to get an output of 10 class (19+)
output = layerConv(output=output, filters=10, size=1, batch_normalization=False, stride=1, pad="same", activation='linear')
output = GlobalAveragePooling2D()(output)
output = Activation('softmax')(output)
#output = cost layer

model = Model(input_img, output)
#print(model.summary())



opt = SGD(lr=1e-2, momentum=0.9, decay=1e-2/epochs)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(X_test, Y_test))
model.fit_generator(imageGene.flow(X_train, Y_train, batch_size=batch_size, shuffle=True, seed=None),
                    steps_per_epoch=len(X_train) // batch_size, epochs=epochs,
                     validation_data=(X_test, Y_test))

Epoch 1/40
234/234 [==============================] - 90s 383ms/step - loss: 1.3362 - acc: 0.5605 - val_loss: 3.7770 - val_acc: 0.2681
Epoch 2/40
234/234 [==============================] - 86s 369ms/step - loss: 0.8717 - acc: 0.7076 - val_loss: 2.0820 - val_acc: 0.4249
Epoch 3/40
231/234 [============================>.] - ETA: 1s - loss: 0.7150 - acc: 0.7552

KeyboardInterrupt: ignored